# SETI CNN using TF and Binary DS

In [1]:
import requests
import json
#import ibmseti
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pickle
import time
#!sudo pip install sklearn
from sklearn.metrics import confusion_matrix

### import dataset reader

In [13]:
!wget --output-document SETI.zip  https://ibm.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
!unzip -o SETI.zip
import SETI

--2017-05-21 12:55:32--  https://ibm.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip [following]
--2017-05-21 12:55:32--  https://ibm.ent.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.26.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.26.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/5DBj3hKre0cn6SpX6NjUPUDHrgD7gcUo9VYDxucBGaNaTivSNnisGQDfT_uQsSrxAY90dkf7XA1Wx09zifZdmve2ugb40IlOGZVrXW_7xeaUB0-muh3sstVptsC-I0W5vBGf3eE4onuQ56-sQ7mt67N2m-GQ0QToLLFyqiIM1TnFWOHFlGMp6IUr0pL7ARsHsT3D8S2_gOFTc0qgafvlvTT1RbAiP5gSA63SfWMMHfagDnJD7IQRUn6X_9WV-ZP_xlIRyh106UQkO_RMMspMY_kdf

### Download data

### Load data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
dataset = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
!ls SETI/SETI_ds_256x2048/

test-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
test-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [16]:
dataset = SETI.read_data_sets('SETI/SETI_ds_768x1024/', one_hot=True, validation_size=0)
dataset.train.num_examples

Extracting SETI/SETI_ds_768x1024/train-images-idx3-ubyte.gz
Extracting SETI/SETI_ds_768x1024/train-labels-idx1-ubyte.gz
Extracting SETI/SETI_ds_768x1024/test-images-idx3-ubyte.gz
Extracting SETI/SETI_ds_768x1024/test-labels-idx1-ubyte.gz


694

In [3]:
dataset.train.images.shape

(55000, 784)

## CNN

In [4]:
# Parameters
learning_rate = 0.0001
#max_training_iters = 500
training_epochs = 3
batch_size = 20
display_step = 100

# Network Parameters

n_classes = 10 # number of possible classifications for the problem
dropout = 0.75 # Dropout, probability to keep units

height = 28 # height of the image in pixels -- 128
width = 28 # width of the image in pixels -- 1536
n_input = width * height # number of pixels in one image  -196608


In [5]:
x  = tf.placeholder(tf.float32, shape=[None, n_input])
y_ = tf.placeholder(tf.float32, shape=[None, n_classes])

In [6]:
x_image = tf.reshape(x, [-1,height,width,1]) 
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

#### Convolutional Layer 1

In [7]:
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs
convolve1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
h_conv1 = tf.nn.relu(convolve1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
layer1= h_pool1
layer1

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

#### Convolutional Layer 2

In [8]:
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64])) #need 64 biases for 64 outputs
convolve2= tf.nn.conv2d(layer1, W_conv2, strides=[1, 1, 1, 1], padding='SAME')+ b_conv2
h_conv2 = tf.nn.relu(convolve2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
layer2= h_pool2
layer2

<tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>

#### Convolutional Layer 3

In [9]:
W_conv3 = tf.Variable(tf.truncated_normal([5, 5, 64, 128], stddev=0.1))
b_conv3 = tf.Variable(tf.constant(0.1, shape=[128])) #need 64 biases for 64 outputs
convolve3= tf.nn.conv2d(layer2, W_conv3, strides=[1, 1, 1, 1], padding='SAME')+ b_conv3
h_conv3 = tf.nn.relu(convolve3)
h_pool3 = tf.nn.max_pool(h_conv3, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME') #max_pool_2x2
layer3= h_pool3
layer3

<tf.Tensor 'MaxPool_2:0' shape=(?, 2, 2, 128) dtype=float32>

#### Convolutional Layer 4

In [10]:
W_conv4 = tf.Variable(tf.truncated_normal([5, 5, 128, 256], stddev=0.1))
b_conv4 = tf.Variable(tf.constant(0.1, shape=[256])) #need 64 biases for 64 outputs
convolve4= tf.nn.conv2d(layer3, W_conv4, strides=[1, 1, 1, 1], padding='SAME')+ b_conv4
h_conv4 = tf.nn.relu(convolve4)
h_pool4 = tf.nn.max_pool(h_conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
layer4= h_pool4
layer4

ValueError: Filter must not be larger than the input: Filter: (5, 5) Input: (2, 2)

#### Fully Connected Layer 1

In [22]:
input_layer = layer2
dim = input_layer.get_shape().as_list()
dim

[None, 7, 7, 64]

In [23]:
dim[1]*dim[2]*dim[3]

3136

In [24]:
dims= dim[1]*dim[2]*dim[3]
nodes1 = 1024
prv_layer_matrix = tf.reshape(input_layer, [-1, dims])
W_fc1 = tf.Variable(tf.truncated_normal([dims, nodes1], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[nodes1])) # need 1024 biases for 1024 outputs
fcl1  = tf.matmul(prv_layer_matrix, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(fcl1) # ???
h_fc1


<tf.Tensor 'Relu_3:0' shape=(?, 1024) dtype=float32>

#### Fully Connected Layer 2

In [25]:
nodes2 = 256
W_fc2 = tf.Variable(tf.truncated_normal([h_fc1.get_shape().as_list()[1], nodes2], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[nodes2])) 
fcl2  = tf.matmul(h_fc1, W_fc2) + b_fc2
h_fc2 = tf.nn.relu(fcl2) # ???
h_fc2

<tf.Tensor 'Relu_4:0' shape=(?, 256) dtype=float32>

In [26]:
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc2, keep_prob)

#### Softmax Layer

In [28]:
W_fc = tf.Variable(tf.truncated_normal([nodes2, n_classes], stddev=0.1)) #1024 neurons
b_fc = tf.Variable(tf.constant(0.1, shape=[n_classes])) # 10 possibilities for classes [0,1,2,3]
fcl = tf.matmul(layer_drop, W_fc) + b_fc
y_CNN= tf.nn.softmax(fcl)

#### Training


In [29]:
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_l4_conv), reduction_indices=[1]))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_CNN, labels=y_))
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

In [30]:
correct_prediction = tf.equal(tf.argmax(y_CNN,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [31]:
# Initializing the variables
init = tf.initialize_all_variables()


In [32]:
ckpt = tf.train.get_checkpoint_state('MNIST/save')
ckpt

model_checkpoint_path: "MNIST/save/model.ckpt-2"
all_model_checkpoint_paths: "MNIST/save/model.ckpt-0"
all_model_checkpoint_paths: "MNIST/save/model.ckpt-1"
all_model_checkpoint_paths: "MNIST/save/model.ckpt-2"

In [33]:
ckpt.model_checkpoint_path

u'MNIST/save/model.ckpt-2'

In [34]:
#!mkdir MNIST/save

In [38]:
!ls -ltr MNIST/save

total 1308528
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users 39298136 May 21 16:04 model.ckpt-5
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users   107522 May 21 16:04 model.ckpt-5.meta
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users 39298136 May 21 16:05 model.ckpt-6
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users   107522 May 21 16:05 model.ckpt-6.meta
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users 39298136 May 21 16:06 model.ckpt-7
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users   107522 May 21 16:06 model.ckpt-7.meta
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users 39298136 May 21 16:07 model.ckpt-8
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users   107522 May 21 16:07 model.ckpt-8.meta
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users 39298136 May 21 16:08 model.ckpt-9
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users   107522 May 21 16:08 model.ckpt-9.meta
-rw-r----- 1 sd22-2e55b7df66e8c3-b01c69100280 users 83744256 May 22 13:58 model.ckpt-1
-rw-

In [37]:
checkpoint_path = 'MNIST/save/model.ckpt'

# Launch the graph
with tf.Session() as sess:

    
    X_test = dataset.test.images
    y_test = dataset.test.labels
    sess.run(init)
    saver = tf.train.Saver(tf.all_variables())
    
    # load previously trained model if appilcable
    ckpt = tf.train.get_checkpoint_state('MNIST/save')
    if ckpt:
        print "loading model: ",ckpt.model_checkpoint_path
        #saver.restore(sess, ckpt.model_checkpoint_path)
    
    
    #step = 0
    num_examples = dataset.train.num_examples
    # Training cycle
    for epoch in range(training_epochs):
        avg_loss = 0.
        avg_accuracy = 0.
        #dataset.shuffle_data()
        total_batch = int(num_examples / batch_size)
        # print total_batch
        # Loop over all batches
        for step in range(total_batch):
            start = time.time()
            x_batch, y_batch = dataset.train.next_batch(batch_size)
            train_step.run(feed_dict={x: x_batch, y_: y_batch, keep_prob: dropout})
            loss, acc = sess.run([cross_entropy, accuracy], feed_dict={x: x_batch,y_: y_batch,keep_prob: 1.})
            #assert( loss < 1000000 ) # make sure it is not NaN or Inf
            #assert( accuracy < 1000000 ) # make sure it is not NaN or Inf
            
            end = time.time()
            
            avg_loss += loss / total_batch
            
            if step % display_step == 0:

                
                # Calculate batch loss and accuracy
                loss, acc = sess.run([cross_entropy, accuracy], feed_dict={x: x_batch,y_: y_batch,keep_prob: 1.})
                #train_accuracy = accuracy.eval(feed_dict={x:x_batch, y_: y_batch,  keep_prob: 0.5})

                test_accuracy = sess.run(accuracy, feed_dict={x: X_test[0:20], y_: y_test[0:20], keep_prob: 1.})

                print("Iter " + str(step) + \
                    ", Training time= " + "{:.5f}".format(end - start) + \
                    ", Minibatch Loss= " +  "{:.6f}".format(loss) +  \
                    ", Training Accuracy= " + "{:.5f}".format(acc)  + \
                    ", Test Accuracy= " + "{:.5f}".format(test_accuracy) )
        
        # save model every 1 epochs
        if epoch >= 0 and epoch % 1 == 0:
            # Save model
            print ("model saved to {}".format(checkpoint_path))
            saver.save(sess, checkpoint_path, global_step = epoch)

            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_loss) 

    print("Optimization Finished!")

    
    
    # Calculate accuracy for test images
    #print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: X_test[0:30], y_: y_test[0:30], keep_prob: 1.}))
        
    # Find the labels of test set
    #y = sess.run(tf.argmax(y_l4_conv,1), feed_dict={x: X_test[0:100], y_: y_test[0:100], keep_prob: 1.})
    
    # lets save kernels
    #kernels_l1 = sess.run(tf.reshape(tf.transpose(W_conv1, perm=[2, 3, 0, 1]),[32,-1]))
    #kernels_l2 = sess.run(tf.reshape(tf.transpose(W_conv2, perm=[2, 3, 0, 1]),[32*64,-1]))

loading model:  MNIST/save/model.ckpt-2
Iter 0, Training time= 0.15281, Minibatch Loss= 2.381561, Training Accuracy= 0.05000, Test Accuracy= 0.25000
Iter 100, Training time= 0.02251, Minibatch Loss= 1.960578, Training Accuracy= 0.55000, Test Accuracy= 0.75000
Iter 200, Training time= 0.04464, Minibatch Loss= 1.744447, Training Accuracy= 0.75000, Test Accuracy= 0.80000
Iter 300, Training time= 0.02412, Minibatch Loss= 1.740589, Training Accuracy= 0.75000, Test Accuracy= 0.95000
Iter 400, Training time= 0.02501, Minibatch Loss= 1.758932, Training Accuracy= 0.70000, Test Accuracy= 0.95000
Iter 500, Training time= 0.02403, Minibatch Loss= 1.724313, Training Accuracy= 0.70000, Test Accuracy= 0.95000
Iter 600, Training time= 0.02412, Minibatch Loss= 1.750386, Training Accuracy= 0.70000, Test Accuracy= 1.00000
Iter 700, Training time= 0.02404, Minibatch Loss= 1.503368, Training Accuracy= 1.00000, Test Accuracy= 1.00000
Iter 800, Training time= 0.02294, Minibatch Loss= 1.592016, Training Accur

KeyboardInterrupt: 

## Evaluation

In [ ]:
y_ = np.argmax(y_test,1)
confusion_matrix(y_, y)

### Viz

In [ ]:
!wget --output-document utils1.py http://deeplearning.net/tutorial/code/utils.py
import utils1
from utils1 import tile_raster_images

In [ ]:
#from utils import tile_raster_images
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
image = Image.fromarray(tile_raster_images(kernels_l1, img_shape=(5, 5) ,tile_shape=(4, 8), tile_spacing=(1, 1)))
### Plot image
plt.rcParams['figure.figsize'] = (18.0, 18.0)
imgplot = plt.imshow(image)
imgplot.set_cmap('gray')  

In [17]:
image = Image.fromarray(tile_raster_images(kernels_l2, img_shape=(5, 5) ,tile_shape=(4, 12), tile_spacing=(1, 1)))
### Plot image
plt.rcParams['figure.figsize'] = (18.0, 18.0)
imgplot = plt.imshow(image)
imgplot.set_cmap('gray')  

NameError: name 'Image' is not defined